# Train a SCCNN

In this notebook, we will create and train a convolutional neural network in the simplicial complex domain, as proposed in the paper by [Yang et. al : Convolutional Learning on Simplicial Complexes (2023)](https://arxiv.org/abs/2301.11163). 

We train the model to perform complex classification using the shrec16 benchmark dataset. 

## Simplicial Complex Convolutional Neural Networks <a href="https://arxiv.org/pdf/2301.11163.pdf">[SCCNN]</a>

SCCNN extends the SCNN to the complex domain by accounting for inter-simplicial connections, i.e., contributions from simplices of adjacent orders. 

For example, we consider SCCNN layers in an SC of order two. At layer $t$, given the inputs on nodes, edges and faces, $\mathbf{h}_{t}^0,\mathbf{h}_{t}^1$ and $\mathbf{h}_{t}^2$, the SCCNN layer contains the following
$$
    \mathbf{h}_{t+1}^1 = \sigma \bigg[ \mathbf{F}_{t,\downarrow} \mathbf{B}_{1}^\top \mathbf{h}_{t}^{0} + \mathbf{F}_{t} \mathbf{h}_t^1 + \mathbf{F}_{t,\uparrow} \mathbf{B}_{2}  \mathbf{h}_t^{2} \bigg] 
$$
where $\mathbf{F}_t$ is the simplicial convolutional filter defined in the edge space, and $\mathbf{F}_{t,\downarrow}$ and $\mathbf{F}_{t,\uparrow}$ are the convolutional filters based on, respectively, only the lower and upper Laplacians. They are given by 
$$
    \mathbf{F}_{t} = {\theta}_t + \sum_{p_d=1}^{P_d} {\theta}_{t,p_d} (\mathbf{L}_{\downarrow,1})^{p_d}  + \sum_{p_u=1}^{P_u} {\theta}_{t,p_u}  (\mathbf{L}_{\uparrow,1})^{p_u} 
$$
$$
    \mathbf{F}_{t,\downarrow} = {\theta}_t + \sum_{p_d=1}^{P_d} {\theta}_{t,p_d} (\mathbf{L}_{\downarrow,1})^{p_d}  
    \text{ and } 
    \mathbf{F}_{t,\uparrow} = {\theta}_t + \sum_{p_u=1}^{P_u} {\theta}_{t,p_u}  (\mathbf{L}_{\uparrow,1})^{p_u} 
$$

Likewise, for the node output, we have 
$$
    \mathbf{h}_{t+1}^0 = \sigma \bigg[ \mathbf{F}_{t} \mathbf{h}_t^0 + \mathbf{F}_{t,\uparrow} \mathbf{B}_{1}  \mathbf{h}_t^{1} \bigg]
$$
where $\mathbf{F}_t$ and $\mathbf{F}_{t,\uparrow}$ are two graph filters essentially. 

For the face output, we have 
$$
    \mathbf{h}_{t+1}^2 = \sigma \bigg[ \mathbf{F}_{t} \mathbf{h}_{t}^{2}  + \mathbf{F}_{t,\downarrow} \mathbf{B}_{2}^\top  \mathbf{h}_t^{1} \bigg]
$$
where $\mathbf{F}_t$ and $\mathbf{F}_{t,\downarrow}$ are two simplicial filters defined in the triangle (face) space. 

In [135]:
import torch
import numpy as np

from toponetx import SimplicialComplex
import toponetx.datasets.graph as graph

from topomodelx.nn.simplicial.sccnn_layer import SCCNNLayer

# Pre-processing

## Import dataset ##

The first step is to import the Karate Club (https://www.jstor.org/stable/3629752) dataset. This is a singular graph with 34 nodes that belong to two different social groups. We will use these groups for the task of node-level binary classification.

We must first lift our graph dataset into the simplicial complex domain.

In [136]:
dataset = graph.karate_club(complex_type="simplicial")
print(dataset)
max_rank = dataset.dim
print(max_rank)

Simplicial Complex with shape (34, 78, 45, 11, 2) and dimension 4
4


### Define Neighborhood Strctures
Get incidence matrices $\mathbf{B}_1,\mathbf{B}_2$ and Hodge Laplacians $\mathbf{L}_0, \mathbf{L}_1$ and $\mathbf{L}_2$.

Note that the original paper considered the weighted versions of these operators. However, the current TOPONETX package does not provide such feature yet.

In [137]:
incidence_1 = dataset.incidence_matrix(rank=1)
incidence_2 = dataset.incidence_matrix(rank=2)

print(f"The incidence matrix B1 has shape: {incidence_1.shape}.")
print(f"The incidence matrix B2 has shape: {incidence_2.shape}.")

The incidence matrix B1 has shape: (34, 78).
The incidence matrix B2 has shape: (78, 45).


In [138]:
laplacian_0  = dataset.hodge_laplacian_matrix(rank=0,weight=True)
laplacian_down_1 = dataset.down_laplacian_matrix(rank=1,weight=True)
laplacian_up_1 = dataset.up_laplacian_matrix(rank=1,weight=True)
laplacian_down_2 = dataset.down_laplacian_matrix(rank=2,weight=True)
laplacian_up_2 = dataset.up_laplacian_matrix(rank=2,weight=True)

laplacian_0  = dataset.adjacency_matrix(rank=0,weight=True)
laplacian_down_1 = dataset.coadjacency_matrix(rank=1,weight=True)
laplacian_up_1 = dataset.adjacency_matrix(rank=1,weight=True)
laplacian_down_2 = dataset.coadjacency_matrix(rank=2,weight=True)
laplacian_up_2 = dataset.adjacency_matrix(rank=2,weight=True)

In [139]:

laplacian_0 = torch.from_numpy(laplacian_0.todense()).to_sparse()
laplacian_down_1 = torch.from_numpy(laplacian_down_1.todense()).to_sparse()
laplacian_up_1 = torch.from_numpy(laplacian_up_1.todense()).to_sparse()
laplacian_down_2 = torch.from_numpy(laplacian_down_2.todense()).to_sparse()
laplacian_up_2 = torch.from_numpy(laplacian_up_2.todense()).to_sparse()

incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
incidence_2 = torch.from_numpy(incidence_2.todense()).to_sparse()

## Import signal ##

Since our task will be node classification, we must retrieve an input signal on the nodes. The signal will have shape $n_\text{nodes} \times$ in_channels, where in_channels is the dimension of each cell's feature. Here, we have in_channels = channels_nodes $ = 34$. This is because the Karate dataset encodes the identity of each of the 34 nodes as a one hot encoder.

In [140]:
"""A function to obtain features based on the input: rank
"""
def get_simplicial_features(dataset,rank):
    if rank == 0: 
        which_feat = "node_feat"
    elif rank == 1:
        which_feat = "edge_feat"
    elif rank == 2:
        which_feat = "face_feat"
    else:
        raise ValueError(f"input dimension must be 0, 1 or 2, because features are supported on nodes, edges and faces") 
    
    x = []
    for _, v in dataset.get_simplex_attributes(which_feat).items():
        x.append(v)
    
    x = torch.tensor(np.stack(x))
    return x

In [141]:
x_0 = get_simplicial_features(dataset,rank=0)
x_1 = get_simplicial_features(dataset,rank=1)
x_2 = get_simplicial_features(dataset,rank=2)

## Define binary labels
We retrieve the labels associated to the nodes of each input simplex. In the KarateClub dataset, two social groups emerge. So we assign binary labels to the nodes indicating of which group they are a part.

We convert the binary labels into one-hot encoder form, and keep the first four nodes' true labels for the purpose of testing.

In [142]:
y = np.array(
    [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        0,
        1,
        1,
        1,
        1,
        0,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
)
y_true = np.zeros((34, 2))
y_true[:, 0] = y
y_true[:, 1] = 1 - y
y_test = y_true[-4:]
y_train = y_true[:30]

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# Create the SCCNN

In [143]:
class SCCNN(torch.nn.Module):
    """SCCNN implementation for complex classification  
    Note: In this task, we can consider the output on any order of simplices for the classification task, which of course can be amended by a readout layer.                                        

    Parameters
    ----------
    in_channels_all: tuple of int
        Dimension of input features on (nodes, edges, faces)
    intermediate_channels_all: tuple of int
        Dimension of features of intermediate layers on (nodes, edges, faces)
    out_channels_all: tuple of int
        Dimension of output features on (nodes, edges, faces)
    conv_order: int
        Order of convolutions, we consider the same order for all convolutions 
    sc_order: int
        SC order 
    n_layers: int
        Numer of layers 

    """
    def __init__(self, 
                 in_channels_all, 
                 intermediate_channels_all, 
                 out_channels_all, 
                 conv_order, sc_order, 
                 aggr_norm=False, update_func=None, 
                 n_layers=2):
        super().__init__()
        # first layer
        # we use an MLP to map the features on simplices of different dimensions to the same dimension   
        self.in_linear_0 = torch.nn.Linear(in_channels_all[0],intermediate_channels_all[0])
        self.in_linear_1 = torch.nn.Linear(in_channels_all[1],intermediate_channels_all[1])
        self.in_linear_2 = torch.nn.Linear(in_channels_all[2],intermediate_channels_all[2])

        layers = []
        for _ in range(n_layers):
            layers.append(
                SCCNNLayer(in_channels=intermediate_channels_all,out_channels=out_channels_all,conv_order=conv_order,sc_order=sc_order,aggr_norm=aggr_norm,update_func=update_func)
            )
    
        self.layers = torch.nn.ModuleList(layers)  

        out_channels_0, out_channels_1, out_channels_2 = out_channels_all
        self.out_linear_0 = torch.nn.Linear(out_channels_0,2)
        

    def forward(self,x_all,laplacian_all,incidence_all):
        """Forward computation. 
        
        Parameters
        ---------
        x: tuple tensors
            (node, edge, face) features 
            each entry shape = [n_simplices, channels]
      
        laplacian: tuple of tensors
            (graph laplacian L0, down edge laplacian L1_d, upper edge laplacian L1_u, face laplacian L2)
            each entry shape = [n_simplices,n_simplices]

        incidence_1: tuple of tensors
            tuple pf order 1 and 2 incidence matrices 
            shape of B1 = [n_nodes, n_edges]
            shape of B2 = [n_edges, n_faces]
    
        Returns
        --------
        _ : tensor
            shape = [n_nodes, 2]
            One-hot labels assigned to nodes.
        """    
        x_0, x_1, x_2 = x_all
        in_x_0 = self.in_linear_0(x_0)
        in_x_1 = self.in_linear_1(x_1)
        in_x_2 = self.in_linear_2(x_2)

        # Forward through SCCNN 
        x_all = (in_x_0,in_x_1,in_x_2)
        for layer in self.layers:
            x_all = layer(x_all,laplacian_all,incidence_all)
        
        """
        We pass the output on the nodes to a linear layer and use that to generate a probability label for nodes
        """
        x_0, _, _ = x_all 
        logits = self.out_linear_0(x_0)
        label = torch.sigmoid(logits)
        
        return label

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

In [144]:
"""Obtain the initial features on all simplices"""
x_all = (x_0,x_1,x_2)

conv_order = 5
in_channels_all = (x_0.shape[-1],x_1.shape[-1],x_2.shape[-1])
intermediate_channels_all = (4,4,4)
out_channels_all = intermediate_channels_all
num_layers = 2

laplacian_all = (laplacian_0,laplacian_down_1,laplacian_up_1,laplacian_down_2,laplacian_up_2)

incidence_all = (incidence_1,incidence_2)

model = SCCNN(in_channels_all=in_channels_all,intermediate_channels_all=intermediate_channels_all,out_channels_all=out_channels_all,conv_order=conv_order,sc_order=max_rank,n_layers=num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [145]:
test_interval = 1
num_epochs = 20
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    optimizer.zero_grad()
    y_hat = model(x_all, laplacian_all, incidence_all)
    loss = torch.nn.functional.binary_cross_entropy(
        y_hat[: len(y_train)].float(), y_train.float()
    )
    epoch_loss.append(loss.item())
    loss.backward()
    optimizer.step()

    y_pred = torch.where(y_hat > 0.5, torch.tensor(1), torch.tensor(0))
    accuracy = (y_pred[-len(y_train) :] == y_train).all(dim=1).float().mean().item()
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {accuracy:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            y_hat_test = model(x_all, laplacian_all, incidence_all)
            y_pred_test = torch.where(
                y_hat_test > 0.5, torch.tensor(1), torch.tensor(0)
            )
            test_accuracy = (
                torch.eq(y_pred_test[-len(y_test) :], y_test)
                .all(dim=1)
                .float()
                .mean()
                .item()
            )
            print(f"Test_acc: {test_accuracy:.4f}", flush=True)

Epoch: 1 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 2 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000


Epoch: 3 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 4 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 5 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 6 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 7 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 8 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 9 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 10 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 11 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 12 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 13 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 14 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 15 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 16 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 17 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 18 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoch: 19 loss: 13.3333 Train_acc: 0.7667
Test_acc: 1.0000
Epoc